### Data structuring for ML - turning a forecasting problem into a prediction problem

This part produces the .csv that is later used in ```ml_modelling.ipynb```, merged from the four datasets.

Steps:
1. Merge Sales and Prices and Calendar Data
2. create long form dataframe (each row has units sold for per item per day)
3. Creating lags of units sold (7, 14, 21) as features, and time based categorical features (i.e. week of month, day of year etc.)

In [1]:
import pandas as pd
import datetime as dt

In [2]:
df_cal = pd.read_csv("../datasets/calendar.csv")
df_prices = pd.read_csv("../datasets/prices.csv")
df_train = pd.read_csv("../datasets/train.csv")
df_sub = pd.read_csv("../datasets/sample_submission.csv")

In the interests of computational efficiency, we take the last date of train data - 6 months before. So we train on 6 months of training data to forecast for the next 21 days.

In [3]:
# NOTE: Change subset_start_date to change size of train dataset
subset_start_date = "2015-10-30"
df_cal_subset = df_cal.set_index("date")
subset_end_date = df_cal.loc[df_cal["d"] == df_sub.columns[-1]]["date"].iloc[0]

df_cal_subset = df_cal_subset.loc[subset_start_date:subset_end_date]
df_cal_subset = df_cal_subset.reset_index()
df_cal_subset.head()

,date,wm_yr_wk,weekday,wday,month,year,d
0,2015-10-30,11539,Friday,7,10,2015,d_1736
1,2015-10-31,11540,Saturday,1,10,2015,d_1737
2,2015-11-01,11540,Sunday,2,11,2015,d_1738
3,2015-11-02,11540,Monday,3,11,2015,d_1739
4,2015-11-03,11540,Tuesday,4,11,2015,d_1740


Not all items have prices in the df_price dataset (as seen from the differing lengths when subsetting df_price by different weeks)

Which introduces NAs in the merged dataset later on for random items on random days. So we can drop these rows with NAs later on.

In [4]:
print(df_prices.loc[df_prices["wm_yr_wk"] == 11547].shape)
print(df_prices.loc[df_prices["wm_yr_wk"] == 11617].shape)

(30466, 4)
(30490, 4)


In [5]:
# Merge prices to subset
df_prices_cal = df_cal_subset.merge(df_prices, on = "wm_yr_wk", how = "left")
df_prices_cal["id"] = df_prices_cal["item_id"].add("_").add(df_prices_cal["store_id"])
df_prices_cal = df_prices_cal.drop(['store_id'], axis=1)

In [6]:
# Return day_ids and dates for the subset
d_to_date_dict_no_forecast = dict(zip(list(df_cal_subset.index[:-21]), list(df_cal_subset["d"][:-21])))
d_to_date_dict = dict(zip(list(df_cal_subset.index), list(df_cal_subset["d"])))

# Create dict to rename columns for submission
sub_dates = pd.date_range(start = "2016-05-01", end = "2016-05-21")
d_to_date = dict(zip(list(df_sub.columns[1:]), list(sub_dates)))
date_to_d = dict(zip(list(sub_dates), list(df_sub.columns[1:])))

In [7]:
# Subset the relevant dates from train.csv
df_train_subset = df_train[list(df_train.columns[:6]) + list(d_to_date_dict_no_forecast.values())]
df_long_train = df_train_subset.melt(id_vars = list(df_train_subset.columns[:6]), value_vars=list(df_train_subset.columns[6:]), var_name = "d", value_name="units_sold")
df_long_train.head()

,id,item_id,subcat_id,category_id,store_id,region_id,d,units_sold
0,Beauty_1_001_East_1,Beauty_1_001,Beauty_1,Beauty,East_1,East,d_1736,2
1,Beauty_1_002_East_1,Beauty_1_002,Beauty_1,Beauty,East_1,East,d_1736,1
2,Beauty_1_003_East_1,Beauty_1_003,Beauty_1,Beauty,East_1,East,d_1736,0
3,Beauty_1_004_East_1,Beauty_1_004,Beauty_1,Beauty,East_1,East,d_1736,0
4,Beauty_1_005_East_1,Beauty_1_005,Beauty_1,Beauty,East_1,East,d_1736,1


In [8]:
# Melt submission dataframe to same format as train
df_sub[["item_id", "category_id", "store_id", "region_id"]] = df_train[["item_id", "category_id", "store_id", "region_id"]]
df_temp = df_sub["item_id"].str.split("_", n = 2, expand = True).iloc[:, :2]
df_sub["subcat_id"] = df_temp[0].str.cat(df_temp[1], sep = "_")
df_sub.head()

,id,d_1920,d_1921,d_1922,d_1923,d_1924,d_1925,d_1926,d_1927,d_1928,...,d_1936,d_1937,d_1938,d_1939,d_1940,item_id,category_id,store_id,region_id,subcat_id
0,Beauty_1_001_East_1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,Beauty_1_001,Beauty,East_1,East,Beauty_1
1,Beauty_1_002_East_1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,Beauty_1_002,Beauty,East_1,East,Beauty_1
2,Beauty_1_003_East_1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,Beauty_1_003,Beauty,East_1,East,Beauty_1
3,Beauty_1_004_East_1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,Beauty_1_004,Beauty,East_1,East,Beauty_1
4,Beauty_1_005_East_1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,Beauty_1_005,Beauty,East_1,East,Beauty_1


In [9]:
df_sub = df_sub.melt(id_vars = ["id", "item_id", "category_id", "store_id", "region_id", "subcat_id"], value_vars = list(df_sub.columns[1:-5]), var_name = "d", value_name = "units_sold")
df_sub.head()

,id,item_id,category_id,store_id,region_id,subcat_id,d,units_sold
0,Beauty_1_001_East_1,Beauty_1_001,Beauty,East_1,East,Beauty_1,d_1920,1
1,Beauty_1_002_East_1,Beauty_1_002,Beauty,East_1,East,Beauty_1,d_1920,1
2,Beauty_1_003_East_1,Beauty_1_003,Beauty,East_1,East,Beauty_1,d_1920,1
3,Beauty_1_004_East_1,Beauty_1_004,Beauty,East_1,East,Beauty_1,d_1920,1
4,Beauty_1_005_East_1,Beauty_1_005,Beauty,East_1,East,Beauty_1,d_1920,1


In [10]:
# Append df_sub to bottom of df_long_train
df_long_train = df_long_train.append(df_sub)

In [11]:
print(df_long_train.dtypes)
df_long_train["id"] = df_long_train["id"].astype("string")
df_long_train["category_id"] = df_long_train["category_id"].astype("string")
df_long_train["item_id"] = df_long_train["item_id"].astype("string")
df_long_train["subcat_id"] = df_long_train["subcat_id"].astype("string")
df_long_train["store_id"] = df_long_train["store_id"].astype("string")
df_long_train["region_id"] = df_long_train["region_id"].astype("string")
df_long_train["d"] = df_long_train["d"].astype("string")

id             object
item_id        object
subcat_id      object
category_id    object
store_id       object
region_id      object
d              object
units_sold      int64
dtype: object


In [12]:
print(df_prices_cal.dtypes)
df_prices_cal["weekday"] = df_prices_cal["weekday"].astype("string")
df_prices_cal["d"] = df_prices_cal["d"].astype("string")
df_prices_cal["id"] = df_prices_cal["id"].astype("string")
df_prices_cal["item_id"] = df_prices_cal["item_id"].astype("string")
df_prices_cal["date"] = pd.to_datetime(df_prices_cal["date"])

date           object
wm_yr_wk        int64
weekday        object
wday            int64
month           int64
year            int64
d              object
item_id        object
sell_price    float64
id             object
dtype: object


In [13]:
# Merge training data with calendar and price data
df_long_train = df_long_train.merge(df_prices_cal, on = ['d', 'id'], how = 'left')
df_long_train.head()

,id,item_id_x,subcat_id,category_id,store_id,region_id,d,units_sold,date,wm_yr_wk,weekday,wday,month,year,item_id_y,sell_price
0,Beauty_1_001_East_1,Beauty_1_001,Beauty_1,Beauty,East_1,East,d_1736,2,2015-10-30,11539.0,Friday,7.0,10.0,2015.0,Beauty_1_001,8.26
1,Beauty_1_002_East_1,Beauty_1_002,Beauty_1,Beauty,East_1,East,d_1736,1,2015-10-30,11539.0,Friday,7.0,10.0,2015.0,Beauty_1_002,3.97
2,Beauty_1_003_East_1,Beauty_1_003,Beauty_1,Beauty,East_1,East,d_1736,0,2015-10-30,11539.0,Friday,7.0,10.0,2015.0,Beauty_1_003,2.97
3,Beauty_1_004_East_1,Beauty_1_004,Beauty_1,Beauty,East_1,East,d_1736,0,2015-10-30,11539.0,Friday,7.0,10.0,2015.0,Beauty_1_004,4.64
4,Beauty_1_005_East_1,Beauty_1_005,Beauty_1,Beauty,East_1,East,d_1736,1,2015-10-30,11539.0,Friday,7.0,10.0,2015.0,Beauty_1_005,2.88


In [14]:
# Remove extra id cols
df_long_train["item_id"] = df_long_train["item_id_x"]
df_long_train = df_long_train.drop(["item_id_x", "item_id_y"], axis = 1)

### Create lags for 7, 14, 21

We created lags in multiples of 7 because of the weekly seasonality we observed in the EDA. So the units sold one week (and more) ago could be good predictors for the current week.

In [15]:
# Take dates from selected start date up till the end of the train dataset
mask = (df_cal['date'] >= subset_start_date) & (df_cal['date'] <= "2016-04-30")
train_subset_d = df_cal.loc[mask]["d"]

# Append sub df to train to capture the shift
df_sub = pd.read_csv("../datasets/sample_submission.csv")
df_lags = df_train.loc[:, ["id"] + list(train_subset_d)]
df_lags[list(df_sub.columns[1:])] = df_sub.iloc[:, 1:]

# Set forecast portion to 0
df_lags[list(df_sub.columns[1:])] = 0

In [16]:
df_train_l21 = df_lags.iloc[:, 1:].shift(21, axis = 1)
df_train_l21["id"] = df_lags["id"]

df_train_l21.head()

,d_1736,d_1737,d_1738,d_1739,d_1740,d_1741,d_1742,d_1743,d_1744,d_1745,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,id
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,1,1,0,0,0,2,0,3,Beauty_1_001_East_1
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,1,0,0,0,0,Beauty_1_002_East_1
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,1,1,0,0,1,1,0,2,Beauty_1_003_East_1
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3,7,2,0,0,1,2,4,1,Beauty_1_004_East_1
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2,2,4,1,0,2,3,1,0,Beauty_1_005_East_1


In [17]:
# Check if the last day of train data has been pushed back by 21 days
assert(df_lags["d_1919"].equals(df_train_l21["d_1940"]))

In [18]:
# Do for lag 7 and 14
df_train_l7 = df_lags.iloc[:, 1:].shift(7, axis = 1)
df_train_l7["id"] = df_lags["id"]

df_train_l14 = df_lags.iloc[:, 1:].shift(14, axis = 1)
df_train_l14["id"] = df_lags["id"]

# Pivot all to match the long form train df
df_train_l7 = df_train_l7.melt(id_vars = "id", value_vars = list(df_train_l7.columns[:-1]), var_name = "d", value_name = "units_sold_l7")
df_train_l14 = df_train_l14.melt(id_vars = "id", value_vars = list(df_train_l14.columns[:-1]), var_name = "d", value_name = "units_sold_l14")
df_train_l21 = df_train_l21.melt(id_vars = "id", value_vars = list(df_train_l21.columns[:-1]), var_name = "d", value_name = "units_sold_l21")

# Fill NA as 0 (NOTE: Is this a good way to deal with the NA?)
df_train_l7["units_sold_l7"] = df_train_l7["units_sold_l7"].fillna(0)
df_train_l14["units_sold_l14"] = df_train_l14["units_sold_l14"].fillna(0)
df_train_l21["units_sold_l21"] = df_train_l21["units_sold_l21"].fillna(0)

df_train_l7["units_sold_l7"] = df_train_l7["units_sold_l7"].astype(int)
df_train_l14["units_sold_l14"] = df_train_l14["units_sold_l14"].astype(int)
df_train_l21["units_sold_l21"] = df_train_l21["units_sold_l21"].astype(int)

In [19]:
# Check that there are exact amount of entries, so there will be no NAs
assert(df_train_l7.shape[0] == df_long_train.shape[0])
assert(df_train_l14.shape[0] == df_long_train.shape[0])
assert(df_train_l21.shape[0] == df_long_train.shape[0])

#### Merge lags into long train df

As the order of the items were retained throughout the merging, we can just append the new lags as new columns in the long train dataset. If not, merging by ID takes very long. 

In [20]:
df_long_train["units_sold_l7"] = df_train_l7["units_sold_l7"]
df_long_train["units_sold_l14"] = df_train_l14["units_sold_l14"]
df_long_train["units_sold_l21"] = df_train_l21["units_sold_l21"]

df_long_train.head()

,id,subcat_id,category_id,store_id,region_id,d,units_sold,date,wm_yr_wk,weekday,wday,month,year,sell_price,item_id,units_sold_l7,units_sold_l14,units_sold_l21
0,Beauty_1_001_East_1,Beauty_1,Beauty,East_1,East,d_1736,2,2015-10-30,11539.0,Friday,7.0,10.0,2015.0,8.26,Beauty_1_001,0,0,0
1,Beauty_1_002_East_1,Beauty_1,Beauty,East_1,East,d_1736,1,2015-10-30,11539.0,Friday,7.0,10.0,2015.0,3.97,Beauty_1_002,0,0,0
2,Beauty_1_003_East_1,Beauty_1,Beauty,East_1,East,d_1736,0,2015-10-30,11539.0,Friday,7.0,10.0,2015.0,2.97,Beauty_1_003,0,0,0
3,Beauty_1_004_East_1,Beauty_1,Beauty,East_1,East,d_1736,0,2015-10-30,11539.0,Friday,7.0,10.0,2015.0,4.64,Beauty_1_004,0,0,0
4,Beauty_1_005_East_1,Beauty_1,Beauty,East_1,East,d_1736,1,2015-10-30,11539.0,Friday,7.0,10.0,2015.0,2.88,Beauty_1_005,0,0,0


#### Create time related features 
I.e. day_of_year is between 1 and 365, week_of_month is between 1 and 4.
These variables are categorical variables, because even though they are integers, the relationship between week 1 and 2 is not the same as between the integers 1 and 2 only.

Creating these features would help to capture time based correlations and seasonalities between the dates and the items sold. For example, if there is a peak of units sold every 7th day of the week, that correlation would be captured by creating these time based features.

In [21]:
df_long_train["date"] = pd.to_datetime(df_long_train["date"])
df_long_train['day_of_month'] = df_long_train['date'].dt.day
df_long_train['day_of_year'] = df_long_train['date'].dt.dayofyear
df_long_train['week_of_month'] = (df_long_train['date'].dt.day - 1) // 7 + 1
df_long_train['week_of_year'] = df_long_train['date'].dt.strftime('%U')

In [22]:
# Clean data before exporting to csv
df_long_train = df_long_train.set_index("date")

#### Uncomment this to export to csv (takes about 2.5 mins to export)

In [25]:
# df_long_train.to_csv("../datasets/long_train_6_mths_with_lags.csv")